In [2]:
import h5py
import hdf5_interface
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, Legend
import requests
from bokeh.palettes import Spectral4


In [3]:
def hdf5_to_dataframe(hdf5_filename, location_name, panel_name):
    """This function extracts data from an HDF5 file and loads it into a pandas dataframe"""
    #Load the HDF5 file data
    hdf5_file = h5py.File('{}.h5'.format(hdf5_filename), 'r')
    hdf5_location = hdf5_file.get(location_name)
    panel_location = hdf5_location.get(panel_name)
    dataframe = pd.DataFrame()
    for keys in panel_location.keys():
        dataframe[str(keys)] = panel_location[str(keys)]
    dataframe['Month'] = dataframe['Month'].astype(int)
    return dataframe


coordinate = pd.DataFrame(columns=['location','latitude','longitude'])
coordinate['location']=['Ambler-Shungnak-Kobuk','Anchorage','Bethel','Chickaloon',
                        'Deering','Denali Park','Fairbanks','Fort Yukon',
                        'Galena-Koyukuk-Ruby', 'Homer','Naknek','Noatak',
                        'Noorvik','Soldotna','Valdez','Wasilla-Palmer']

coordinate['latitude']=[66.995834, 61.193625, 60.794938, 61.823570, 
                        66.069413, 63.537277, 64.838033, 66.571563,
                        64.782991, 59.652521, 58.728349, 67.570921, 
                        66.836039, 60.486370, 61.128663, 61.582242]


coordinate['longitude']=[ -157.377096, -149.694974, -161.770716, -148.450442,
                         -162.766760,  -150.985453, -147.668970,  -145.250173,
                         -156.744933, -151.536496, -157.017444, -162.967490,
                          -161.041913, -151.060702, -146.353366, -149.441001]

def pvwatts_tmy2(lattitude,longitude):
    # Get the data from the PV Watts --TMY2
    list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 18, "module_type": 0, "losses": 14.08,
                  "array_type": 0, "tilt": 50, "azimuth": 180, "lat": lattitude, "lon": longitude, "dataset": 'tmy2'}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    TMY2 = pd.DataFrame(data = json_response['outputs'])
    return TMY2

def pvwatts_tmy3(lattitude,longitude):
    # Get the data from the PV Watts --TMY3
    list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 18, "module_type": 0, "losses": 14.08,
                  "array_type": 0, "tilt": 50, "azimuth": 180, "lat": lattitude, "lon": longitude, "dataset": 'tmy3'}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    TMY3 = pd.DataFrame(data = json_response['outputs'])
    return TMY3

def annual_Norm(dataframe,capacity):
    '''Calculate rolling annual production'''
    lenth_list = list(range(12,len(dataframe.index)))
    annual_values = []
    month = []
    for i in range(len(lenth_list)):
        single_values = dataframe['Energy'][lenth_list[i]-12:lenth_list[i]].sum()/capacity
        #rolling_average.append(each_period)
        single_month = dataframe['Date'][lenth_list[i]]
        annual_values.append(single_values)
        month.append(single_month)
    return(annual_values,month)

def daily_to_monthly_energy(file_name, location_name, panel_name):
    solar_dataframe = hdf5_to_dataframe(file_name, location_name, panel_name)
    new_dataframe = pd.DataFrame(columns = ['Year', 'Month', 'Energy', 'Interpolate'])
    previous_month_tracker = solar_dataframe['Month'][0]
    Sum = 0
    interpolated = 0
    j = 0 
    year_array = np.array(np.NaN)
    month_array = np.array(np.NaN)
    interpolation_array = np.array(np.NaN)
    energy_array = np.array(np.NaN)
    for i in range(len(solar_dataframe.index)):
        if solar_dataframe['Month'][i] == previous_month_tracker:
            Sum = Sum + solar_dataframe['Energy'][i]
            interpolated = interpolated + solar_dataframe['Interpolate'][i]
        else:
            year_array = np.append(year_array, solar_dataframe['Year'][i])
            month_array = np.append(month_array, solar_dataframe['Month'][i])
            if interpolated > 0:
                interpolation_array = np.append(interpolation_array, 1)
            else:
                interpolation_array = np.append(interpolation_array, 0)
            energy_array = np.append(energy_array, Sum)
            Sum = 0
            interpolated = 0
            j = j + 1
            previous_month_tracker = solar_dataframe['Month'][i]
    new_dataframe['Energy'] = energy_array.astype(int)
    new_dataframe['Month'] = month_array.astype(int)
    new_dataframe['Year'] = year_array.astype(int)
    new_dataframe['Interpolate'] = interpolation_array.astype(int)
    new_dataframe = new_dataframe.drop(0).reset_index(drop=True)
    
    #new_dataframe['DC Capacity'][1] = solar_dataframe['DC Capacity'][0]
    #new_dataframe['Location'][1] = solar_dataframe['Location'][0]
    return new_dataframe

In [4]:
my_file = h5py.File("../../GEGU/solar_panel_data_alaska.h5", 'r')

In [ ]:
location=pd.DataFrame(columns=['Month', 'Year'])
fairbanks = my_file.get('Fairbanks')
a=0
for name in fairbanks:
    capacity = fairbanks[name].attrs.__getitem__("DC Capacity")
    base = hdf5_to_dataframe('../../GEGU/solar_panel_data_alaska','Fairbanks', name )
    base['Energy'] = base['Energy']/capacity
    location = pd.merge(location, base, on = ['Month', 'Year'], how='outer',suffixes=(a, a+1))
    a = a+1
    
# # set up a dataframe to store TMY2&3 ac_monthly
# pv = pd.DataFrame()
# pv['TMY2'] = TMY2.ac_monthly
# pv['TMY3'] = TMY3.ac_monthly
# pv['Month'] = [1,2,3,4,5,6,7,8,9,10,11,12]

# result = pd.merge(location, pv, on = ['Month'], how='outer' )
# #merge PVWatts data into location data


# result['Date'] = ""
# # Change the date into a datetime format
# for i in range(len(result)):
#     result['Date'][i] = str(result['Year'][i]) + '-' + str(result['Month'][i])    
#     result['Date'] = pd.to_datetime(result['Date'])


# result = result.sort_values(by='Date')
# # sort by date

# result